In [1]:
#EDA

In [2]:
import torch

In [3]:
#!pip install pysentimiento transformers datasets accelerate evaluate

In [4]:
import datasets
import evaluate

In [5]:
import numpy as np
from datasets import load_dataset

In [6]:
import ipywidgets as widgets

In [7]:
#Helper Functions

In [8]:
# label to name
def label2name(x):
    if x == 0:
        return "Negative"
    if x == 1:
        return "Neutral"
    if x == 2:
        return "Positive"

In [9]:

f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")

def compute_metrics (eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)
    
    results = {}
    results.update(f1_metric.compute(predictions=preds, references = labels, average="macro"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="macro"))
    return results

In [10]:
#Read Data


In [11]:
data_files = {"train": "data/train.csv", "validation": "data/val.csv", "test": "data/test.csv"}
ds = load_dataset("csv", data_files=data_files)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [12]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count'],
        num_rows: 878
    })
    validation: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count'],
        num_rows: 109
    })
    test: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count'],
        num_rows: 98
    })
})

In [13]:
ds["train"].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'label_name': Value(dtype='string', id=None),
 'tokenized_text': Value(dtype='string', id=None),
 'sent_token_length': Value(dtype='int64', id=None),
 'sent_bert_token_length': Value(dtype='int64', id=None),
 'char_count': Value(dtype='int64', id=None),
 'Character Count': Value(dtype='int64', id=None)}

In [14]:
ds["test"]["label"][:10]

[2, 1, 1, 0, 1, 1, 0, 0, 0, 1]

In [15]:
df = ds['train'].to_pandas()
df.head()

,text,label,label_name,tokenized_text,sent_token_length,sent_bert_token_length,char_count,Character Count
0,"Yo soy Misterio, saco mi fierro y te mato",0,Negative,"Yo soy Misterio, saco mi fierro y te mato",9,10,41,41
1,"Unos bandidos, unos negros con caras de forajidos",1,Neutral,"Unos bandidos, unos negros con caras de forajidos",8,12,49,49
2,-Quieres decir que soy un soplón -dijo el Jaguar,1,Neutral,-Quieres decir que soy un soplón -dijo el Jaguar,9,14,49,49
3,soy recontra taco pero soy bien rica,2,Positive,soy recontra taco pero soy bien rica,7,7,36,36
4,"El colegio estaba cerrado, todavía era temprano",1,Neutral,"El colegio estaba cerrado, todavía era temprano",7,8,47,47


In [16]:
# encode label and mapping label name
#df["label"] = df["label"].apply(lambda x: label_encode(x))
df["label_name"] = df["label"].apply(lambda x: label2name(x))

In [17]:
# clean text, lowercase and remove punk
#df["text"] = df["text"].apply(lambda x: remove_punct(clean(remove_emoji(x).lower())[0][0]))

In [18]:
df.head()

,text,label,label_name,tokenized_text,sent_token_length,sent_bert_token_length,char_count,Character Count
0,"Yo soy Misterio, saco mi fierro y te mato",0,Negative,"Yo soy Misterio, saco mi fierro y te mato",9,10,41,41
1,"Unos bandidos, unos negros con caras de forajidos",1,Neutral,"Unos bandidos, unos negros con caras de forajidos",8,12,49,49
2,-Quieres decir que soy un soplón -dijo el Jaguar,1,Neutral,-Quieres decir que soy un soplón -dijo el Jaguar,9,14,49,49
3,soy recontra taco pero soy bien rica,2,Positive,soy recontra taco pero soy bien rica,7,7,36,36
4,"El colegio estaba cerrado, todavía era temprano",1,Neutral,"El colegio estaba cerrado, todavía era temprano",7,8,47,47


In [19]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "pysentimiento/robertuito-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 128

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/435M [00:00<?, ?B/s]

Some weights of the model checkpoint at pysentimiento/robertuito-base-uncased were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-uncased and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.den

In [20]:
# columns = ['input_ids', 'attention_mask', 'label']
# ds.set_format(type='torch', columns=columns)
# ds

In [21]:
from pysentimiento.preprocessing import preprocess_tweet
preprocessed_ds = ds.map(lambda ex: {"text": preprocess_tweet(ex["text"], lang="es")})

Map:   0%|          | 0/878 [00:00<?, ? examples/s]

Map:   0%|          | 0/109 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In [22]:
tokenized_ds = preprocessed_ds.map(
    lambda batch: tokenizer(
        batch["text"], padding=True, truncation=True
        ),
    batched=True, batch_size=32
)

Map:   0%|          | 0/878 [00:00<?, ? examples/s]

Map:   0%|          | 0/109 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In [23]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 878
    })
    validation: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 109
    })
    test: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 98
    })
})

In [24]:
tokenized_ds['train']['input_ids'][0]

[0,
 560,
 977,
 10692,
 16,
 5713,
 507,
 27084,
 445,
 516,
 7521,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [25]:
tokenized_ds['train']['attention_mask'][0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [26]:
tokenized_ds['train']['text'][0]

'Yo soy Misterio, saco mi fierro y te mato'

In [27]:
#!pip install ipdb

In [28]:
#import torch
#device = "cuda" if torch.cuda.is_available() else "cpu"
#model.cuda()

In [29]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

training_args = TrainingArguments(
    per_device_train_batch_size=32,
    output_dir="test_trainer",
    do_eval=True,
    evaluation_strategy="epoch",
    num_train_epochs=5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [30]:
trainer.train()

C:\Users\JairL\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'eval_loss': 0.8017800450325012, 'eval_f1': 0.4876152832674572, 'eval_recall': 0.5289115646258503, 'eval_runtime': 5.7787, 'eval_samples_per_second': 18.862, 'eval_steps_per_second': 2.423, 'epoch': 1.0}
{'eval_loss': 0.7035928964614868, 'eval_f1': 0.7109988776655443, 'eval_recall': 0.681895485466914, 'eval_runtime': 5.674, 'eval_samples_per_second': 19.211, 'eval_steps_per_second': 2.467, 'epoch': 2.0}
{'eval_loss': 0.6875613927841187, 'eval_f1': 0.7162697896334852, 'eval_recall': 0.6951530612244898, 'eval_runtime': 5.6518, 'eval_samples_per_second': 19.286, 'eval_steps_per_second': 2.477, 'epoch': 3.0}
{'eval_loss': 0.7171606421470642, 'eval_f1': 0.6822053176845614, 'eval_recall': 0.6667439703153989, 'eval_runtime': 5.796, 'eval_samples_per_second': 18.806, 'eval_steps_per_second': 2.415, 'epoch': 4.0}
{'eval_loss': 0.6991589069366455, 'eval_f1': 0.7267223160591744, 'eval_recall': 0.7099567099567099, 'eval_runtime': 5.8878, 'eval_samples_per_second': 18.513, 'eval_steps_per_second':

TrainOutput(global_step=140, training_loss=0.4795149666922433, metrics={'train_runtime': 814.8538, 'train_samples_per_second': 5.387, 'train_steps_per_second': 0.172, 'train_loss': 0.4795149666922433, 'epoch': 5.0})

In [31]:
#Test on validation set

In [32]:
trainer.evaluate(tokenized_ds["test"])

{'eval_loss': 0.8051414489746094, 'eval_f1': 0.6862993604412368, 'eval_recall': 0.6643939393939394, 'eval_runtime': 2.7952, 'eval_samples_per_second': 35.06, 'eval_steps_per_second': 4.651, 'epoch': 5.0}


{'eval_loss': 0.8051414489746094,
 'eval_f1': 0.6862993604412368,
 'eval_recall': 0.6643939393939394,
 'eval_runtime': 2.7952,
 'eval_samples_per_second': 35.06,
 'eval_steps_per_second': 4.651,
 'epoch': 5.0}

In [33]:
ds['test']['text']

['Te estoy llevando al mejor troca de Lima',
 'Debe ser jodido trabajar en la bolsa',
 'Siguieron dando vueltas, sin hablar, fumando',
 'llora conchatumadre\x85\x94 ¿Ves? Ya me muero carajo',
 'Desde tiempos viejos, la batalla había sido dura',
 'Tengo tanta hambre que me comería una vaca cruda',
 'No le hagas caso a la bestia de tu papá',
 '\x97¿Qué te pasa oye? ¿Por qué estás así todo achorado?',
 '\x97No, eso se fue al carajo',
 'Se han superado una inmensidad estos cocodrilos',
 '\x97Apúrate, gordo rosquete',
 'Nada de maricón',
 '\x97Como quieras, chibolo',
 'El día que ellos vengan, tú zafas al toque',
 '\x97Anda a buscar a tu esposa, canalla \x97dijo Joaquín',
 'A veces las lisuras llegaban hasta la cocina y Símula cerraba la puerta',
 '\x97Ay, pobre, qué piña',
 'Vinimos al instante, casi choco en la Costanera',
 'Yo voy a cuidar tu castidad como oro en polvo',
 'mañana tengo que chambear',
 'CARADURA:  No voy a morir como un cabro',
 'Me volví a quemar, maldita sea',
 'Eres un

In [34]:
#model.load_state_dict(torch.load(f'./_BERT_epoch_3.model', map_location=torch.device('cpu')))

In [35]:
#Error Analisys

In [36]:
val_df = ds['validation'].to_pandas()
#val_df

In [37]:
from tqdm.notebook import tqdm

In [38]:
# step by step predictions on dataframe
# We do this to view predictions in the pandas dataframe and easily filter them and perform error analysis.
pred_final = []

# for i, row in tqdm(val_df.iterrows(), total=val_df.shape[0]):
#     predictions = []

#     text = row["text"]
#     encoded_data_test_single = tokenizer.batch_encode_plus([text], 
#     add_special_tokens=config.add_special_tokens, 
#     return_attention_mask=config.return_attention_mask, 
#     pad_to_max_length=config.pad_to_max_length, 
#     max_length=config.seq_length,
#     return_tensors=config.return_tensors
#     )
#     input_ids_test = encoded_data_test_single['input_ids']
#     attention_masks_test = encoded_data_test_single['attention_mask']

    
#     inputs = {'input_ids':      input_ids_test.to(device),
#               'attention_mask':attention_masks_test.to(device),
#              }

#     with torch.no_grad():        
#         outputs = model(**inputs)
    
#     logits = outputs[0]
#     logits = logits.detach().cpu().numpy()
#     predictions.append(logits)
#     predictions = np.concatenate(predictions, axis=0)
#     pred_final.append(np.argmax(predictions, axis=1).flatten()[0])

In [39]:
df.sample(n = 30)

,text,label,label_name,tokenized_text,sent_token_length,sent_bert_token_length,char_count,Character Count
331,"No necesitaba sacar la chaveta, se lo juro",1,Neutral,"No necesitaba sacar la chaveta, se lo juro",8,10,42,42
247,¿O sea que tú nunca has visto la calata de Ca...,1,Neutral,¿O sea que tú nunca has visto la calata de Ca...,11,13,52,52
789,¿Sabes que Ambrosio se fue? dijo el Chispas,1,Neutral,¿Sabes que Ambrosio se fue? dijo el Chispas,8,11,46,46
316,¿Quién es el Chacal? Un pata,1,Neutral,¿Quién es el Chacal? Un pata,6,10,30,30
215,"Pero la negra bonita, la picarona, no contestó",0,Negative,"Pero la negra bonita, la picarona, no contestó",8,9,46,46
680,No seas maricona,0,Negative,No seas maricona,3,4,16,16
585,Se acabaron los negocios y ahora empieza el chupe,1,Neutral,Se acabaron los negocios y ahora empieza el chupe,9,9,49,49
462,Pero lo más raro del caso es que la perra se curó,0,Negative,Pero lo más raro del caso es que la perra se curó,12,13,49,49
814,"Escúchame bien, serrano conchatumadre",0,Negative,"Escúchame bien, serrano conchatumadre",4,8,37,37
39,"cuidado chibolo, tienes que aprender a respetar",0,Negative,"cuidado chibolo, tienes que aprender a respetar",7,9,47,47


In [40]:
#Inference

In [41]:
#Evaluate the Model Qualitatively (Human Evaluation)

In [42]:
#Evaluate the Model Quantitatively (with F1 Metric)

In [43]:
texts = ds['test'][0:10]['text']
human_baseline_labels = ds['test'][0:10]['label']

In [44]:
texts

['Te estoy llevando al mejor troca de Lima',
 'Debe ser jodido trabajar en la bolsa',
 'Siguieron dando vueltas, sin hablar, fumando',
 'llora conchatumadre\x85\x94 ¿Ves? Ya me muero carajo',
 'Desde tiempos viejos, la batalla había sido dura',
 'Tengo tanta hambre que me comería una vaca cruda',
 'No le hagas caso a la bestia de tu papá',
 '\x97¿Qué te pasa oye? ¿Por qué estás así todo achorado?',
 '\x97No, eso se fue al carajo',
 'Se han superado una inmensidad estos cocodrilos']

In [45]:
human_baseline_labels

[2, 1, 1, 0, 1, 1, 0, 0, 0, 1]

In [46]:
#original_model_summaries = []
model_classifications = []

In [47]:
for _, text in enumerate(texts):
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to('cuda')
    logits = model(input_ids).logits
    probabilities = logits.softmax(dim=-1).tolist()[0]
    model_classifications.append(np.argmax(probabilities).flatten()[0])

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
model_classifications